### k-NN method based on dijsktra short-path algorithm metric

In [1]:
import numpy as np
import pandas as pd

**Save data**

In [2]:
df = pd.read_csv("data.csv", index_col=False)

**Sample data**

In [3]:
df.head()

,arr_line,dest_line,arr_station,dest_station
0,Сокольническая линия,Сокольническая линия,Черкизовская,Преображенская площадь
1,Сокольническая линия,Сокольническая линия,Преображенская площадь,Сокольники
2,Сокольническая линия,Сокольническая линия,Сокольники,Красносельская
3,Сокольническая линия,Сокольническая линия,Красносельская,Комсомольская
4,Сокольническая линия,Сокольническая линия,Комсомольская,Красные ворота


In [4]:
df["arr_line"] = " " + df["arr_line"]

**Transform data to uniform view**

In [5]:
new_df = pd.DataFrame(columns=['from', 'to'])
new_df['from'] = df['arr_line'] + df['arr_station']
new_df['to'] = df['dest_line'] + df['dest_station']
new_df.head()

,from,to
0,Сокольническая линия Черкизовская,Сокольническая линия Преображенская площадь
1,Сокольническая линия Преображенская площадь,Сокольническая линия Сокольники
2,Сокольническая линия Сокольники,Сокольническая линия Красносельская
3,Сокольническая линия Красносельская,Сокольническая линия Комсомольская
4,Сокольническая линия Комсомольская,Сокольническая линия Красные ворота


**Create the adjustment matrix**

In [6]:
stations = np.unique([*new_df['from'], *new_df['to']])
dim = stations.shape[0]
adj_matrix = np.zeros([dim, dim], dtype='int')
print(adj_matrix)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


**Create dictionaries for storing names and keys**

In [7]:
dict_stations = dict(zip(stations, range(stations.shape[0]))) 
rev_dict_stations = dict(zip(range(stations.shape[0]), stations)) 

**New dataframe with keys**

In [8]:
df_num = new_df.copy()
df_num['from'] = df_num['from'].map(dict_stations)
df_num['to'] = df_num['to'].map(dict_stations)
df_num.head()

,from,to
0,164,158
1,158,160
2,160,152
3,152,151
4,151,153


**Fill the adjustment matrix**

In [9]:
list_from = list(df_num['from'])
list_to = list(df_num['to'])
for i in range(len(list_from)):
    adj_matrix[list_to[i]][list_from[i]] = adj_matrix[list_from[i]][list_to[i]] = 1

print(adj_matrix)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


**Class "Graph" calculates paths between stations via Dijkstra's short path algorithm**

In [10]:
class Graph:
    def __init__(self, matrix):
        self.matrix = matrix
        self.dim = matrix.shape[0]

    def _short_path(self, src: int):
        dist = np.full(self.dim, np.inf)
        prev = np.full(self.dim, False)
        dist[src] = 0
        for i in range(self.dim):
            u = np.inf
            for j in range(self.dim):
                if prev[j] == False and dist[j] <= u:
                    u = dist[j]
                    cur = j
        
            prev[cur] = True
       
            for j in range(self.dim):
                if self.matrix[cur][j] > 0 and prev[j] == False:
                    if dist[cur] + self.matrix[cur][j] < dist[j]:
                        dist[j] = dist[cur] + self.matrix[cur][j]
        return dist

    def distance_matrix(self):
        return np.array([self._short_path(i) for i in range(self.dim)], dtype='int')

**Calculate distance matrix**

In [11]:
%%time
graph = Graph(adj_matrix)
distances = graph.distance_matrix()
print(distances)

[[ 0  3 10 ...  4  6  8]
 [ 3  0 13 ...  7  9 11]
 [10 13  0 ... 10 10  8]
 ...
 [ 4  7 10 ...  0  2  4]
 [ 6  9 10 ...  2  0  2]
 [ 8 11  8 ...  4  2  0]]
CPU times: user 16.7 s, sys: 103 ms, total: 16.8 s
Wall time: 17.4 s


**Save the resulting matrix into .csv format**

In [12]:
temp = pd.DataFrame(distances)
temp.to_csv("distance_data.csv", index = False)
df_dist = pd.read_csv("distance_data.csv")
df_dist.head()   

,0,1,2,3,4,5,6,7,8,9,...,192,193,194,195,196,197,198,199,200,201
0,0,3,10,7,2,7,5,2,11,6,...,4,3,6,5,5,7,3,4,6,8
1,3,0,13,4,5,10,8,1,14,9,...,7,6,9,8,8,10,6,7,9,11
2,10,13,0,17,8,3,5,12,1,4,...,10,9,6,11,11,7,10,10,10,8
3,7,4,17,0,9,14,12,5,18,13,...,11,10,13,12,12,14,10,11,13,15
4,2,5,8,9,0,5,3,4,9,4,...,2,1,4,3,3,5,2,2,4,6


**Create new dataframe with a list of stations and 'tea-coffee' features**

In [13]:
df_names = pd.DataFrame(stations, columns=['name'])
df_names['coffee'] = 0
df_names['tea'] = 0

In [14]:
tea_map = {' Замоскворецкая линия Царицыно': 2, ' Таганско-Краснопресненская линия Текстильщики': 1, '  Замоскворецкая линия Войковская': 2, ' Калужско-Рижская линия Бабушкинская': 1, ' Серпуховско-Тимирязевская линия Отрадное': 1, ' Замоскворецкая линия Орехово': 1, ' Замоскворецкая линия Ховрино': 1}
coffee_map = {' Калужско-Рижская линия Академическая': 1, ' Серпуховско-Тимирязевская линия Тимирязевская': 1, ' Люблинская линия Окружная': 1, ' Арбатско-Покровская линия Молодёжная': 1, '  Люблинская линия Бутырская': 1, ' Кольцевая линия Комсомольская': 1, ' Замоскворецкая линия Речной вокзал': 1, ' Замоскворецкая линия Войковская': 2, ' Замоскворецкая линия Сокол': 1}
print(coffee_map, tea_map, sep= '\n\n')

{' Калужско-Рижская линия Академическая': 1, ' Серпуховско-Тимирязевская линия Тимирязевская': 1, ' Люблинская линия Окружная': 1, ' Арбатско-Покровская линия Молодёжная': 1, '  Люблинская линия Бутырская': 1, ' Кольцевая линия Комсомольская': 1, ' Замоскворецкая линия Речной вокзал': 1, ' Замоскворецкая линия Войковская': 2, ' Замоскворецкая линия Сокол': 1}

{' Замоскворецкая линия Царицыно': 2, ' Таганско-Краснопресненская линия Текстильщики': 1, '  Замоскворецкая линия Войковская': 2, ' Калужско-Рижская линия Бабушкинская': 1, ' Серпуховско-Тимирязевская линия Отрадное': 1, ' Замоскворецкая линия Орехово': 1, ' Замоскворецкая линия Ховрино': 1}


**fill it and save to .csv**

In [15]:
for i, row in df_names[df_names['name'].isin(coffee_map.keys())].iterrows():
    df_names.at[i, 'coffee'] = coffee_map[df_names.at[i, 'name']]
for i, row in df_names[df_names['name'].isin(tea_map.keys())].iterrows():
    df_names.at[i, 'tea'] = tea_map[df_names.at[i, 'name']]
df_names.to_csv('station_list.csv', index=False)
df_names.head(10)

,name,coffee,tea
0,Арбатско-Покровская линия Арбатская,0,0
1,Арбатско-Покровская линия Бауманская,0,0
2,Арбатско-Покровская линия Волоколамская,0,0
3,Арбатско-Покровская линия Измайловская,0,0
4,Арбатско-Покровская линия Киевская,0,0
5,Арбатско-Покровская линия Крылатское,0,0
6,Арбатско-Покровская линия Кунцевская,0,0
7,Арбатско-Покровская линия Курская,0,0
8,Арбатско-Покровская линия Митино,0,0
9,Арбатско-Покровская линия Молодёжная,1,0


**Add feature columns to the adjustment matrix**

In [16]:
df_names = pd.read_csv('station_list.csv')
df_dist['coffee'] = df_names['coffee']
df_dist['tea'] = df_names['tea']
df_dist.head(10)

,0,1,2,3,4,5,6,7,8,9,...,194,195,196,197,198,199,200,201,coffee,tea
0,0,3,10,7,2,7,5,2,11,6,...,6,5,5,7,3,4,6,8,0,0
1,3,0,13,4,5,10,8,1,14,9,...,9,8,8,10,6,7,9,11,0,0
2,10,13,0,17,8,3,5,12,1,4,...,6,11,11,7,10,10,10,8,0,0
3,7,4,17,0,9,14,12,5,18,13,...,13,12,12,14,10,11,13,15,0,0
4,2,5,8,9,0,5,3,4,9,4,...,4,3,3,5,2,2,4,6,0,0
5,7,10,3,14,5,0,2,9,4,1,...,3,8,8,4,7,7,7,5,0,0
6,5,8,5,12,3,2,0,7,6,1,...,1,6,6,2,5,5,5,3,0,0
7,2,1,12,5,4,9,7,0,13,8,...,8,7,7,9,5,6,8,10,0,0
8,11,14,1,18,9,4,6,13,0,5,...,7,12,12,8,11,11,11,9,0,0
9,6,9,4,13,4,1,1,8,5,0,...,2,7,7,3,6,6,6,4,1,0


**Distances to stations with a filled feature column**

In [17]:
df_tea = df_dist[df_dist['tea'] != 0]
df_coffee = df_dist[df_dist['coffee'] != 0]
df_y = df_dist[(df_dist['tea'] != 0) | (df_dist['coffee'] != 0)]
df_y.head()

,0,1,2,3,4,5,6,7,8,9,...,194,195,196,197,198,199,200,201,coffee,tea
9,6,9,4,13,4,1,1,8,5,0,...,2,7,7,3,6,6,6,4,1,0
35,9,10,16,14,8,13,11,9,17,12,...,12,10,10,13,9,9,11,13,2,0
44,10,11,20,15,12,17,15,10,21,16,...,16,14,14,17,13,13,15,17,0,1
46,11,12,18,16,10,15,13,11,19,14,...,14,12,12,15,11,11,13,15,1,0
47,8,9,15,13,7,12,10,8,16,11,...,11,9,9,12,8,8,10,12,1,0


**The weighted k-nearest neighbour classifier**

$$ Q_{j}=\sum _{{i=1}}^{n}{\frac  {1}{d(x,a_{i})^{2}}} - \text{metric used}$$

In [18]:
class KNN():
    def __init__(self, dict_stations, rev_dict_stations, coffee, tea, y, k=3):
        self.st = dict_stations.copy()
        self.st_keys = rev_dict_stations.copy()
        self.coffee = coffee
        self.tea = tea
        self.y = y
        self.k = k

    def guess_class(self, x: int, k: int)->str:    
        nearest_k = self.y.sort_values(by=str(x)).head(k)
        nearest_k['coffee_weights'] = nearest_k['coffee'] * 1 / (nearest_k[str(x)] ** 2)
        nearest_k['tea_weights'] = nearest_k['tea'] * 1 / (nearest_k[str(x)] ** 2)
        coffee_coef = nearest_k['coffee_weights'].sum()
        tea_coef = nearest_k['tea_weights'].sum()
        
        if coffee_coef > tea_coef:
            return 'coffee'
        else:
            return 'tea'
    
    def on_station(self, station: str)->str:
        temp = self.st[station]
        if temp in self.y.all(1):
            if self.y.loc[temp, :]['coffee'] > self.y.loc[temp, :]['tea']:
                str_drink = 'coffee'
            else:
                str_drink = 'tea'
        else:
            str_drink = self.guess_class(temp, self.k)
            
        return 'on "{}" drink {}'.format(self.st_keys[temp], str_drink)

**Tests calculating class with 5 neighbors**

In [19]:
knn = KNN(dict_stations, rev_dict_stations, df_coffee, df_tea, df_y, 5)

In [20]:
result = knn.on_station(' Замоскворецкая линия Войковская')
print(result)
result1 = knn.on_station(' Замоскворецкая линия Алма-Атинская')
print(result1)

on " Замоскворецкая линия Войковская" drink coffee
on " Замоскворецкая линия Алма-Атинская" drink tea


In [21]:
from pprint import pprint
res = enumerate([knn.on_station(station) for station in stations])
with open('results.txt', 'w') as file:
    pprint(dict(res), stream=file)